In [5]:
import os
from rich.console import Console
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()
console = Console()

# 1. 기본 Usage 정보 확인 과정

In [12]:
MODEL_NAME = "claude-3-5-haiku-20241022"

client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),  # This is the default and can be omitted
)

message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Hello, Claude",
        }
    ],
    model=MODEL_NAME,
)

console.log(message)

[13:11:27] Message(                                                                                ]8;id=900275;file:///tmp/ipykernel_4160277/2455806266.py\2455806266.py]8;;\:]8;id=529154;file:///tmp/ipykernel_4160277/2455806266.py#18\18]8;;\
               id='msg_01HXLXZrXV6fFBKDUQ41fZZR',                                                                  
               content=[                                                                                           
                   TextBlock(                                                                                      
                       citations=None,                                                                             
                       text='Hello! How are you doing today?',                                                     
                       type='text'                                                                                 
                   )                                                                                               
               ],                                                                                                  
               model='claude-3-5-haiku-20241022',                                                                  
               role='assistant',                                                                                   
               stop_reason='end_turn',                                                                             
               stop_sequence=None,                                                                                 
               type='message',                                                                                     
               usage=Usage(                                                                                        
                   cache_creation_input_tokens=0,                                                                  
                   cache_read_input_tokens=0,                                                                      
                   input_tokens=10,                                                                                
                   output_tokens=11,                                                                               
                   server_tool_use=None,                                                                           
                   service_tier='standard'                                                                         
               )                                                                                                   
           )                                                                                                       

# 2. 일반 텍스트 캐싱 테스트

아래 코드는 캐싱이 발생하지 않는다.

In [26]:
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Hello, Claude",
                    "cache_control": {"type": "ephemeral"},
                },
            ]
        }
    ],
    model=MODEL_NAME,
)

console.log(message)

[13:49:12] Message(                                                                                ]8;id=587651;file:///tmp/ipykernel_4160277/2969606964.py\2969606964.py]8;;\:]8;id=773196;file:///tmp/ipykernel_4160277/2969606964.py#18\18]8;;\
               id='msg_01Hd4GTPViD11rq3iTw4gwM8',                                                                  
               content=[                                                                                           
                   TextBlock(                                                                                      
                       citations=None,                                                                             
                       text='Hello! How are you doing today?',                                                     
                       type='text'                                                                                 
                   )                                                                                               
               ],                                                                                                  
               model='claude-3-5-haiku-20241022',                                                                  
               role='assistant',                                                                                   
               stop_reason='end_turn',                                                                             
               stop_sequence=None,                                                                                 
               type='message',                                                                                     
               usage=Usage(                                                                                        
                   cache_creation_input_tokens=0,                                                                  
                   cache_read_input_tokens=0,                                                                      
                   input_tokens=10,                                                                                
                   output_tokens=11,                                                                               
                   server_tool_use=None,                                                                           
                   service_tier='standard'                                                                         
               )                                                                                                   
           )                                                                                                       

아래 코드는 캐싱이 발생해야 한다. Haiku 모델은 2048 토큰 이상만 캐싱되기 때문에 임의로 긴 텍스트를 만들어서 전송함.

In [31]:
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Hello, Claude" * 400,
                    "cache_control": {"type": "ephemeral"},
                },
            ]
        }
    ],
    model=MODEL_NAME,
)

console.log(message)

[14:14:34] Message(                                                                                ]8;id=819778;file:///tmp/ipykernel_4160277/3216904136.py\3216904136.py]8;;\:]8;id=313561;file:///tmp/ipykernel_4160277/3216904136.py#18\18]8;;\
               id='msg_01Hc8dmY89GW6fSQ9xiLPvYL',                                                                  
               content=[                                                                                           
                   TextBlock(                                                                                      
                       citations=None,                                                                             
                       text='Hello! It seems like you\'ve sent multiple "Hello, Claude" messages.                  
           Is there something specific I can help you with today?',                                                
                       type='text'                                                                                 
                   )                                                                                               
               ],                                                                                                  
               model='claude-3-5-haiku-20241022',                                                                  
               role='assistant',                                                                                   
               stop_reason='end_turn',                                                                             
               stop_sequence=None,                                                                                 
               type='message',                                                                                     
               usage=Usage(                                                                                        
                   cache_creation_input_tokens=2400,                                                               
                   cache_read_input_tokens=0,                                                                      
                   input_tokens=4,                                                                                 
                   output_tokens=30,                                                                               
                   server_tool_use=None,                                                                           
                   service_tier='standard'                                                                         
               )                                                                                                   
           )                                                                                                       

재실행하면 캐싱된 토큰을 사용한다.

In [32]:
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Hello, Claude" * 400,
                    "cache_control": {"type": "ephemeral"},
                },
            ]
        }
    ],
    model=MODEL_NAME,
)

console.log(message)

[14:14:44] Message(                                                                                ]8;id=770088;file:///tmp/ipykernel_4160277/3216904136.py\3216904136.py]8;;\:]8;id=584174;file:///tmp/ipykernel_4160277/3216904136.py#18\18]8;;\
               id='msg_01SteeBPE7rDrTmqLAfozF4e',                                                                  
               content=[                                                                                           
                   TextBlock(                                                                                      
                       citations=None,                                                                             
                       text='Hello! It seems like you\'ve sent me the word "Hello, Claude" many                    
           times. Is there something specific I can help you with today?',                                         
                       type='text'                                                                                 
                   )                                                                                               
               ],                                                                                                  
               model='claude-3-5-haiku-20241022',                                                                  
               role='assistant',                                                                                   
               stop_reason='end_turn',                                                                             
               stop_sequence=None,                                                                                 
               type='message',                                                                                     
               usage=Usage(                                                                                        
                   cache_creation_input_tokens=0,                                                                  
                   cache_read_input_tokens=2400,                                                                   
                   input_tokens=4,                                                                                 
                   output_tokens=33,                                                                               
                   server_tool_use=None,                                                                           
                   service_tier='standard'                                                                         
               )                                                                                                   
           )                                                                                                       

새로운 메세지를 추가해도 과거에 캐싱한 데이터가 있으면 캐싱은 여전히 사용된다.

In [33]:
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Hello, Claude" * 400,
                },
            ]
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": "네 반갑습니당. 근데 왜케 길게 보내셨",
                    "cache_control": {"type": "ephemeral"},
                },
            ]
        }
    ],
    model=MODEL_NAME,
)

console.log(message)

[14:15:58] Message(                                                                                ]8;id=263763;file:///tmp/ipykernel_4160277/1033580577.py\1033580577.py]8;;\:]8;id=515958;file:///tmp/ipykernel_4160277/1033580577.py#27\27]8;;\
               id='msg_01Wqywf9XGhoboimSwVYYL6g',                                                                  
               content=[TextBlock(citations=None, text='죠? 😄', type='text')],                                    
               model='claude-3-5-haiku-20241022',                                                                  
               role='assistant',                                                                                   
               stop_reason='end_turn',                                                                             
               stop_sequence=None,                                                                                 
               type='message',                                                                                     
               usage=Usage(                                                                                        
                   cache_creation_input_tokens=28,                                                                 
                   cache_read_input_tokens=2400,                                                                   
                   input_tokens=1,                                                                                 
                   output_tokens=9,                                                                                
                   server_tool_use=None,                                                                           
                   service_tier='standard'                                                                         
               )                                                                                                   
           )                                                                                                       

In [34]:
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Hello, Claude" * 400,
                },
            ]
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": "네 반갑습니당. 근데 왜케 길게 보내셨죠? 😄",
                    "cache_control": {"type": "ephemeral"},
                },
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "ㅋ 그냥용, 무슨 문제라도?",
                },
            ]
        },
    ],
    model=MODEL_NAME,
)

console.log(message)

[14:17:48] Message(                                                                                ]8;id=817600;file:///tmp/ipykernel_4160277/3114874855.py\3114874855.py]8;;\:]8;id=583534;file:///tmp/ipykernel_4160277/3114874855.py#36\36]8;;\
               id='msg_01F2fCCCxJwZsdXqNbKoMdup',                                                                  
               content=[                                                                                           
                   TextBlock(                                                                                      
                       citations=None,                                                                             
                       text='아니요, 특별한 문제는 없어요. 다만 반복되는 "Hello, Claude" 메시지가                  
           너무 많아서 살짝 재미있었어요. 무엇을 도와드릴까요?',                                                   
                       type='text'                                                                                 
                   )                                                                                               
               ],                                                                                                  
               model='claude-3-5-haiku-20241022',                                                                  
               role='assistant',                                                                                   
               stop_reason='end_turn',                                                                             
               stop_sequence=None,                                                                                 
               type='message',                                                                                     
               usage=Usage(                                                                                        
                   cache_creation_input_tokens=34,                                                                 
                   cache_read_input_tokens=2400,                                                                   
                   input_tokens=28,                                                                                
                   output_tokens=73,                                                                               
                   server_tool_use=None,                                                                           
                   service_tier='standard'                                                                         
               )                                                                                                   
           )                                                                                                       

In [36]:
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Hello, Claude" * 400,
                },
            ]
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": "네 반갑습니당. 근데 왜케 길게 보내셨죠? 😄",
                    # "cache_control": {"type": "ephemeral"},
                },
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "ㅋ 그냥용, 무슨 문제라도?",
                },
            ]
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": "문제는 없죠 ㅋ, 근데 제 생각에는",
                    "cache_control": {"type": "ephemeral"},
                },
            ]
        },
    ],
    model=MODEL_NAME,
)

console.log(message)

[14:20:07] Message(                                                                                ]8;id=776698;file:///tmp/ipykernel_4160277/2995487554.py\2995487554.py]8;;\:]8;id=215921;file:///tmp/ipykernel_4160277/2995487554.py#46\46]8;;\
               id='msg_014Hbkch9BAZdqob5RZxAQmg',                                                                  
               content=[                                                                                           
                   TextBlock(                                                                                      
                       citations=None,                                                                             
                       text=' 같은 메시지를 반복해서 보내는 건 별 의미가 없어 보여요. 혹시 제가                    
           도와드릴 일이 있나요?',                                                                                 
                       type='text'                                                                                 
                   )                                                                                               
               ],                                                                                                  
               model='claude-3-5-haiku-20241022',                                                                  
               role='assistant',                                                                                   
               stop_reason='end_turn',                                                                             
               stop_sequence=None,                                                                                 
               type='message',                                                                                     
               usage=Usage(                                                                                        
                   cache_creation_input_tokens=0,                                                                  
                   cache_read_input_tokens=2486,                                                                   
                   input_tokens=1,                                                                                 
                   output_tokens=57,                                                                               
                   server_tool_use=None,                                                                           
                   service_tier='standard'                                                                         
               )                                                                                                   
           )                                                                                                       

# 3. 이미지/PDF 포함 캐싱 테스트

In [44]:
import base64

# Prepare image and PDF files
image_path = "/home/lsw91/capture.png"
pdf_path = "/home/lsw91/test.pdf"

# Convert image to base64
with open(image_path, "rb") as image_file:
    image_data = image_file.read()
    base64_image = base64.b64encode(image_data).decode("utf-8")

# Convert PDF to base64
with open(pdf_path, "rb") as pdf_file:  
    pdf_data = pdf_file.read()
    base64_pdf_data = base64.b64encode(pdf_data).decode("utf-8")

# Create API blocks for image and PDF
api_image_block = {
    "type": "image",
    "source": {
        "type": "base64",
        "media_type": "image/png",
        "data": base64_image,
    },
    "cache_control": {"type": "ephemeral"}
}   

api_pdf_block = {
    "type": "document",
    "source": {
        "type": "base64",
        "media_type": "application/pdf",
        "data": base64_pdf_data,
    },
    "cache_control": {"type": "ephemeral"}
}

In [42]:
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                api_image_block,
                api_pdf_block,
                {
                    "type": "text",
                    "text": "위 문서에 있는 내용들을 요약해줘.",
                    "cache_control": {"type": "ephemeral"},
                },
            ]
        }
    ],
    model=MODEL_NAME,
)

console.log(message)

[17:17:11] Message(                                                                                 ]8;id=400973;file:///tmp/ipykernel_4160277/728419379.py\728419379.py]8;;\:]8;id=43876;file:///tmp/ipykernel_4160277/728419379.py#20\20]8;;\
               id='msg_01MCobaTWY2TDtUipXu1sL3t',                                                                  
               content=[                                                                                           
                   TextBlock(                                                                                      
                       citations=None,                                                                             
                       text='이 문서는 다결정 재료의 결정 방위를 분석하는 장치와 방법에 관한 일본                  
           특허 문서입니다. 주요 내용은 다음과 같습니다:\n\n1. 목적\n- 다결정 재료(예: 다결정                      
           실리콘 기판)의 결정 방위를 간편하고 효율적으로 분석하는 기술 개발\n\n2. 기존 문제점\n-                  
           기존 X선이나 전자선을 이용한 결정 방위 측정 방법은 진공 환경, 긴 측정 시간, 대형 장비                   
           필요 등의 한계 존재\n\n3. 제안된 분석 방법\n- 시료 표면에 대해 다양한 방위각과 입사각을                 
           가진 조명광을 조사\n- 조명광의 방위각을 변화시키며 시료 표면의 휘도 화상을 촬상\n- 휘도                 
           화상의 각 화소 휘도값을 벡터로 변환\n- 머신러닝 모델을 이용해 결정 방위를 추정\n\n4.                    
           주요 기술적 특징\n- 회전 쿼터니온을 이용해 결정 방위 표현\n- 리커런트 신경망(RNN) 기반                  
           학습 모델 사용\n- 조명 조건에 따른 휘도 변화를 통해 결정 방위 추정\n\n5. 장점\n- 진공                   
           챔버 없이 대기 중에서 측정 가능\n- 측정 시간 단축\n- 시료 표면 전체의 결정 방위를                       
           신속하게 분석 가능\n\n이 기술은 특히 태양전지 제조 분야에서 다결정 실리콘 기판의 품질                   
           관리에 유용할 것으로 기대됩니다.',                                                                      
                       type='text'                                                                                 
                   )                                                                                               
               ],                                                                                                  
               model='claude-3-5-haiku-20241022',                                                                  
               role='assistant',                                                                                   
               stop_reason='end_turn',                                                                             
               stop_sequence=None,                                                                                 
               type='message',                                                                                     
               usage=Usage(                                                                                        
                   cache_creation_input_tokens=60242,                                                              
                   cache_read_input_tokens=0,                                                                      
                   input_tokens=0,                                                                                 
                   output_tokens=578,                                                                              
                   server_tool_use=None,                                                                           
                   service_tier='standard'                                                                         
               )                                                                                                   
           )                                                                                                       

In [45]:
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                api_image_block,
                api_pdf_block,
                {
                    "type": "text",
                    "text": "위 문서에 있는 내용들을 요약해줘.",
                    "cache_control": {"type": "ephemeral"},
                },
            ]
        }
    ],
    model=MODEL_NAME,
)

console.log(message)

[17:18:50] Message(                                                                                 ]8;id=146269;file:///tmp/ipykernel_4160277/728419379.py\728419379.py]8;;\:]8;id=995128;file:///tmp/ipykernel_4160277/728419379.py#20\20]8;;\
               id='msg_01UbUsbgFLtYbjKjSQZMuBDh',                                                                  
               content=[                                                                                           
                   TextBlock(                                                                                      
                       citations=None,                                                                             
                       text='이 문서는 결정 방위 분석 장치 및 방법에 관한 일본 특허 문서입니다.                    
           주요 내용을 요약하면 다음과 같습니다:\n\n1. 목적\n- 다결정 재료(예: 다결정 실리콘                       
           기판)의 결정 방위를 간편하고 효율적으로 분석하는 기술 제공\n\n2. 주요 장치 구성\n- 측정                 
           장치: \n  * 시료 표면에 특정 입사각과 방위각을 가진 조명광 다발을 조사\n  * 조명광                      
           다발의 방위각을 변화시키며 시료 표면 촬상\n- 계산 장치:\n  * 촬상된 복수의 휘도 이미지                  
           분석\n  * 결정립의 분포와 결정 방위 추정\n\n3. 분석 방법\n- 조명광 다발의 방위각 변화에                 
           따른 시료 표면 휘도 변화 측정\n- 휘도 벡터 생성 및 클러스터링을 통한 결정립 분포 추정\n-                
           기계학습 모델을 이용한 결정 방위 추정\n\n4. 주요 특징\n- 진공 챔버나 X선/전자선 없이                    
           광학적 측정 가능\n- 회전 쿼터니온을 이용해 결정 방위 표현\n- 딥러닝 기반 학습 모델로                    
           결정 방위 추정\n\n5. 장점\n- 측정 시간 단축\n- 대면적 시료의 전수 검사 용이\n- 비파괴                   
           분석 가능\n- 높은 공간 분해능\n\n이 기술은 태양전지 등 다결정 재료의 품질 평가에 활용될                 
           수 있습니다.',                                                                                          
                       type='text'                                                                                 
                   )                                                                                               
               ],                                                                                                  
               model='claude-3-5-haiku-20241022',                                                                  
               role='assistant',                                                                                   
               stop_reason='end_turn',                                                                             
               stop_sequence=None,                                                                                 
               type='message',                                                                                     
               usage=Usage(                                                                                        
                   cache_creation_input_tokens=0,                                                                  
                   cache_read_input_tokens=60242,                                                                  
                   input_tokens=0,                                                                                 
                   output_tokens=568,                                                                              
                   server_tool_use=None,                                                                           
                   service_tier='standard'                                                                         
               )                                                                                                   
           )                                                                                                       